In [1]:
from numpy import array

def split_sequence(sequence,n_steps_in,n_steps_out):
  X,y = list() , list()

  for i in range(len(sequence)):
    end_ix = i+n_steps_in
    out_end_ix = end_ix +n_steps_out

    if out_end_ix > len(sequence):
      break

    seq_X,seq_y = sequence[i:end_ix],sequence[end_ix:out_end_ix]
    X.append(seq_X)
    y.append(seq_y)

  return array(X), array(y)



In [2]:
raw_seq = [10,20,30,40,50,60,70,80,90]
n_steps_in,n_steps_out = 3,2

X,y =split_sequence(raw_seq,n_steps_in,n_steps_out)

for i in range(len(X)):
  print(X[i],y[i])


[10 20 30] [40 50]
[20 30 40] [50 60]
[30 40 50] [60 70]
[40 50 60] [70 80]
[50 60 70] [80 90]


2============

In [3]:
from keras.models import Sequential
from keras.layers import LSTM,Dense,RepeatVector,TimeDistributed

In [4]:

n_features = 1
X=X.reshape((X.shape[0], X.shape[1],n_features))


model = Sequential()
model.add(LSTM(100,activation='relu', return_sequences=True, input_shape=(n_steps_in,n_features)))
model.add(LSTM(100,activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss = 'mse')

model.fit(X,y,epochs=50, verbose=0)

x_input= array([70,80,90])
x_input = x_input.reshape(1,n_steps_in,n_features)
yhat = model.predict(x_input, verbose=0)

print(yhat)


[[125.325134 140.21336 ]]


3====================

In [ ]:
n_features = 1
X=X.reshape((X.shape[0], X.shape[1],n_features))
y=y.reshape((y.shape[0], y.shape[1],n_features))

model = Sequential()
model.add(LSTM(100,activation='relu', input_shape=(n_steps_in,n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(100,activation='relu',return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss = 'mse')

model.fit(X,y,epochs=100, verbose=0)

x_input= array([70,80,90])
x_input = x_input.reshape(1,n_steps_in,n_features)
yhat = model.predict(x_input, verbose=0)

print(yhat)


[[[102.99554]
  [117.51076]]]


4==========================

In [17]:
from numpy import hstack

in_seq1 = array([10,20,30,40,50,60,70,80,90])
in_seq2 = array([15,25,35,45,55,65,75,85,95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

in_seq1 = in_seq1.reshape(len(in_seq1), 1 )
in_seq2 = in_seq2.reshape(len(in_seq2), 1 )
out_seq = out_seq.reshape(len(out_seq), 1 )

dataset = hstack((in_seq1,in_seq2,out_seq))

n_steps_in,n_steps_out= 3,2


X,y =split_sequence(dataset,n_steps_in,n_steps_out)
print(X.shape, y.shape)

for i in range(len(x1)):
  print('\n',X[i])
  print('\n',y[i])




(5, 3, 3) (5, 2, 3)

 [[10 15 25]
 [20 25 45]
 [30 35 65]]

 [[ 40  45  85]
 [ 50  55 105]]

 [[20 25 45]
 [30 35 65]
 [40 45 85]]

 [[ 50  55 105]
 [ 60  65 125]]

 [[ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]]

 [[ 60  65 125]
 [ 70  75 145]]

 [[ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]]

 [[ 70  75 145]
 [ 80  85 165]]

 [[ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]]

 [[ 80  85 165]
 [ 90  95 185]]


Hàm cắt data

In [31]:

def split_sequence2(sequence,n_steps_in,n_steps_out):
  X,y = list() , list()

  for i in range(len(sequence)):
    end_ix = i+n_steps_in
    out_end_ix = end_ix +n_steps_out-1

    if out_end_ix > len(sequence):
      break

    seq_X,seq_y = sequence[i:end_ix,:-1],sequence[end_ix-1:out_end_ix,-1]
    X.append(seq_X)
    y.append(seq_y)

  return array(X), array(y)


Tạo data

In [32]:
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])

in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))

dataset = hstack((in_seq1, in_seq2, out_seq))

n_steps_in, n_steps_out = 3, 2

X, y = split_sequence2(dataset, n_steps_in, n_steps_out)

In [34]:
n_features = X.shape[2]
model = Sequential()
model.add(LSTM(100,activation='relu',return_sequences=True, input_shape=(n_steps_in,n_features)))
model.add(LSTM(100,activation='relu'))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss = 'mse')


model.fit(X,y,epochs=200, verbose=0)

x_input= array([[70,75],[80,85],[90,95]])
x_input = x_input.reshape(1,n_steps_in,n_features)
yhat = model.predict(x_input, verbose=0)

print(yhat)

[[188.42558 210.31982]]


======================================

In [ ]:
n_features = X.shape[2]
model = Sequential()
model.add(LSTM(200,activation='relu', input_shape=(n_steps_in,n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(200,activation='relu',return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss = 'mse')


model.fit(X,y,epochs=300, verbose=0)

x_input= array([[60,65,125],[70,75,145],[80,85,165]])
x_input = x_input.reshape(1,n_steps_in,n_features)
yhat = model.predict(x_input, verbose=0)

print(yhat)



[[[ 90.071945  94.962975 185.30492 ]
  [ 99.9012   105.134895 205.27377 ]]]
